In [102]:
import pandas as pd
import numpy as np
from preprocessing import transform_csv
from sklearn.model_selection import train_test_split
from sklearn import svm, preprocessing
import pickle

In [153]:
scale_feat = ['Age', 'Fare']

def fit_wrap(X, y, clf):
    X_toscale = X[scale_feat]
    scaler = preprocessing.StandardScaler().fit(X_toscale)
#     X.loc[:, scale_feat] = scaler.transform(X_toscale)
    clf.fit(X, y)
    return {'classifier': clf, 'scaler': scaler}

def predict_wrap(X, model):
#     X.loc[:, scale_feat] = model['scaler'].transform(X[scale_feat])
    y_hat = model['classifier'].predict(X)
    return y_hat

def train_accuracy(X_train, y_train, X_test, y_test, clf):
    model = fit_wrap(X_train, y_train, clf)
    y_hat = predict_wrap(X_test, model)
    return 1 - np.average(np.abs(y_hat-y_test))

def submission(X, y, passengers, X_sub, clf, output='submission.csv'):
    model = fit_wrap(X, y, clf)
    y_sub = predict_wrap(X_sub, model)
    submission = pd.DataFrame()
    submission['PassengerId'] = passengers
    submission['Survived'] = y_sub
    submission.to_csv(output, index=False)

In [154]:
transform_csv('train.csv', 'train_processed.csv')
train_df = pd.read_csv('train_processed.csv')
y = train_df['Survived']
passengers = train_df['PassengerId']
X = train_df.drop(['Survived', 'PassengerId'], axis=1)

In [155]:
transform_csv('test.csv', 'test_processed.csv')
test_df = pd.read_csv('test_processed.csv')
passengers_sub = test_df['PassengerId']
X_sub = test_df.drop(['PassengerId'], axis=1)

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

1. Vanilla SVM classifier with sklearn

In [158]:
clf_svm = svm.SVC(gamma=0.001, C=100)
print(train_accuracy(X_train, y_train, X_test, y_test, clf_svm))

0.815642458101


In [159]:
clf_svm = svm.SVC(gamma=0.001, C=100)
submission(X, y, passengers_sub, X_sub, clf_svm, output='svm_submission.csv')

2. Linear Regression

In [161]:
from sklearn import linear_model
clf_lr = linear_model.LinearRegression()
print(train_accuracy(X_train, y_train, X_test, y_test, clf_lr))

-131989437.777
